In [1]:
! pip install gpt4all

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
base_url = os.environ.get('OPENAI_BASE_URL')
api_key = os.environ.get('OPENAI_API_KEY')
model_name = os.environ.get('OPENAI_MODEL_NAME')

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=api_key, model_name=model_name, base_url=base_url)

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path= "be-good-and-how-not-to-die.txt")

In [5]:
document = loader.load()

In [6]:
print(type(document))

<class 'list'>


In [7]:
print(len(document))

1


In [8]:
print(document[0].metadata)

{'source': 'be-good-and-how-not-to-die.txt'}


In [9]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 27423 characters


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n', '\n\n'],
    chunk_size=3000,
    chunk_overlap=400
)

In [11]:
documents = text_splitter.split_documents(document)

In [28]:
documents[0]

Document(page_content="Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We've\nlearned a lot since then, but if I were choosing now that's still\nthe one I'd pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it's so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon't worry too much about making money.  What you've got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren't supposed to be\nlike charities, and we've proven by reductio ad absurdum that one\nor both of the principles we began with is false

In [29]:
documents[1]

Document(page_content="realized it would probably have to be a company.  It would cost\nsomething to run, and it would be a pain to fund with grants and\ndonations.That was a surprising realization.  Companies often claim to be\nbenevolent, but it was surprising to realize there were purely\nbenevolent projects that had to be embodied as companies to work.I didn't want to start another company, so I didn't do it.  But if\nsomeone had, they'd probably be quite rich now.  There was a window\nof about two years when spam was increasing rapidly but all the big\nemail services had terrible filters.  If someone had launched a\nnew, spam-free mail service, users would have flocked to it.Notice the pattern here?  From either direction we get to the same\nspot.  If you start from successful startups, you find they often\nbehaved like nonprofits.  And if you start from ideas for nonprofits,\nyou find they'd often make good startups.PowerHow wide is this territory?  Would all good nonprofits be g

In [25]:
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings as Embeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.vectorstores.redis import Redis
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embedding = Embeddings(model_name=model_name, gpt4all_kwargs=gpt4all_kwargs)
stored_embeddings = Chroma.from_documents(documents = documents,
                                          embedding= embedding,
                                          collection_name='rag_chroma'
                                          )

In [26]:
from langchain_community.chains import PebbloRetrievalQA

QA_chain = PebbloRetrievalQA.from_chain_type(llm = llm, 
                                             chain_type= "stuff",
                                             retriever = stored_embeddings.as_retriever())

ValidationError: 1 validation error for PebbloRetrievalQA
retriever
  Vectorstore must be an instance of one of the supported vectorstores: [<class 'langchain_community.vectorstores.pinecone.Pinecone'>, <class 'langchain_community.vectorstores.qdrant.Qdrant'>]. Got Chroma instead. (type=value_error)

In [ ]:
question = """
What is this article about?
Describe it in less than 100 words.
"""

In [ ]:
QA_chain.invoke(question)

In [ ]:
question2 = """
And how does it explain how to create something people want?
"""

In [ ]:
QA_chain.invoke(question2)